In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 09 - Green operators, kernels, convolution\n",
        "\n",
        "We validate a basic convolution adjoint pairing identity for circular convolution.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import tensorflow as tf\n",
        "from tig.core.random import Rng\n",
        "\n",
        "tf.keras.backend.set_floatx(\"float64\")\n",
        "tf.config.run_functions_eagerly(True)\n",
        "\n",
        "def circ_conv_real(x: tf.Tensor, h: tf.Tensor) -> tf.Tensor:\n",
        "    xx = tf.cast(tf.reshape(x, (-1,)), tf.complex128)\n",
        "    hh = tf.cast(tf.reshape(h, (-1,)), tf.complex128)\n",
        "    y = tf.signal.ifft(tf.signal.fft(xx) * tf.signal.fft(hh))\n",
        "    return tf.cast(tf.math.real(y), tf.float64)\n",
        "\n",
        "def circ_conv_adjoint_real(y: tf.Tensor, h: tf.Tensor) -> tf.Tensor:\n",
        "    yy = tf.cast(tf.reshape(y, (-1,)), tf.complex128)\n",
        "    hh = tf.cast(tf.reshape(h, (-1,)), tf.complex128)\n",
        "    hhat = tf.signal.fft(hh)\n",
        "    x = tf.signal.ifft(tf.signal.fft(yy) * tf.math.conj(hhat))\n",
        "    return tf.cast(tf.math.real(x), tf.float64)\n",
        "\n",
        "rng = Rng(seed=0)\n",
        "n = 512\n",
        "x = rng.normal((n,), dtype=tf.float64)\n",
        "y = rng.normal((n,), dtype=tf.float64)\n",
        "h = rng.normal((n,), dtype=tf.float64)\n",
        "\n",
        "Ax = circ_conv_real(x, h)\n",
        "Aty = circ_conv_adjoint_real(y, h)\n",
        "\n",
        "left = tf.reduce_sum(Ax * y)\n",
        "right = tf.reduce_sum(x * Aty)\n",
        "gap = tf.abs(left - right)\n",
        "denom = tf.abs(left) + tf.abs(right) + tf.cast(1e-15, tf.float64)\n",
        "\n",
        "print(\"relative pairing gap:\", float((gap / denom).numpy()))\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.x"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
